In [1]:
import random
import vizdoom as vd
import numpy as np
import scipy.ndimage as Simg

from basic_ennemy_pos import basic_ennemy_x
from network import tf, DRQN
from video import VideoWriter
from memory import ReplayMemory
from config import (
    N_ACTIONS, LEARNING_RATE, MIN_MEM_SIZE, MAX_MEM_SIZE,
    MAX_CPUS, TRAINING_STEPS, BATCH_SIZE, SEQUENCE_LENGTH,
    QLEARNING_STEPS, MAX_EPISODE_LENGTH, DEATH_PENALTY,
    KILL_REWARD, PICKUP_REWARD, GREEDY_STEPS, IGNORE_UP_TO,
    BACKPROP_STEPS, USE_GAME_FEATURES, LEARN_Q, USE_RECURRENCE,
)

# Config variables
im_w, im_h = 108, 60
N_FEATURES = 1
ACTION_SET = np.eye(N_ACTIONS, dtype=np.uint32).tolist()
SECTION_SEPARATOR = "------------"

In [2]:
from vizdoom import *
game = vd.DoomGame()
game.load_config("scenarios/deadly_corridor.cfg")
# game.load_config("basic.cfg")
game.set_screen_format(ScreenFormat.RGB24)
game.set_depth_buffer_enabled(True)

# Ennemy detection
walls = None  # map_parser.parse("maps/deathmatch.txt")
game.clear_available_game_variables()
game.add_available_game_variable(vd.GameVariable.POSITION_X)  # 0
game.add_available_game_variable(vd.GameVariable.POSITION_Y)  # 1
game.add_available_game_variable(vd.GameVariable.POSITION_Z)  # 2

game.add_available_game_variable(vd.GameVariable.KILLCOUNT)   # 3
game.add_available_game_variable(vd.GameVariable.DEATHCOUNT)  # 4
game.add_available_game_variable(vd.GameVariable.ITEMCOUNT)   # 5

game.set_labels_buffer_enabled(True)


game.init()

In [3]:
game.new_episode()
dump = []
zoomed = np.zeros((MAX_EPISODE_LENGTH, im_h, im_w, 3), dtype=np.uint8)
action = ACTION_SET[0]
while not game.is_episode_finished():
    # Get screen buf
    state = game.get_state()
    S = state.screen_buffer  # NOQA

    # Resample to our network size
    h, w = S.shape[:2]
    Simg.zoom(S, [1. * im_h / h, 1. * im_w / w, 1],
              output=zoomed[len(dump)], order=0)
    S = zoomed[len(dump)]  # NOQA

    # Get game features an action
    game_features = [basic_ennemy_x(state)]
    action = random.choice(ACTION_SET)
    reward = game.make_action(action, skip)
    dump.append((S, action, reward, game_features))

NameError: name 'skip' is not defined

In [8]:
a = state.depth_buffer

In [10]:
a

In [11]:
state.screen_buffer

array([[[19, 43, 51],
        [19, 43, 51],
        [35, 51, 71],
        ..., 
        [27, 47, 67],
        [27, 47, 67],
        [19, 35, 55]],

       [[27, 47, 67],
        [19, 43, 51],
        [19, 35, 55],
        ..., 
        [19, 35, 55],
        [19, 35, 55],
        [19, 35, 55]],

       [[27, 47, 67],
        [27, 47, 67],
        [19, 35, 55],
        ..., 
        [19, 35, 55],
        [19, 35, 55],
        [19, 35, 55]],

       ..., 
       [[19, 19, 19],
        [19, 19, 19],
        [11, 11, 11],
        ..., 
        [47, 47, 47],
        [47, 47, 47],
        [55, 55, 55]],

       [[19, 19, 19],
        [27, 27, 27],
        [19, 19, 19],
        ..., 
        [47, 47, 47],
        [47, 47, 47],
        [47, 47, 47]],

       [[11, 11, 11],
        [19, 19, 19],
        [19, 19, 19],
        ..., 
        [27, 27, 27],
        [19, 19, 19],
        [19, 19, 19]]], dtype=uint8)

In [11]:
output_shape

NameError: name 'output_shape' is not defined

In [13]:
np.shape(S)

(3, 240, 320)

In [14]:
[1. * im_h / h, 1. * im_w / w, 1]

[20.0, 0.45, 1]

In [15]:
im_h

60

In [16]:
im_w

108

In [17]:
h

3

In [18]:
w

240

In [19]:
S.shape

(3, 240, 320)